In [1]:
! pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 4.3 MB/s eta 0:00:0000:0100:01


In [3]:
import random
import cv2
import numpy as np
from math import ceil

In [4]:
def my_rand(a,b):
    if a<b:
        return random.randint(a,b)
    else:
        return a
current_epoch=0
class RandomBranch:
    class ColorGetter:
        def __init__(self, center_path, std_path):
            self.center_path = center_path
            self.std_path = std_path
            self.centers = np.load(self.center_path)
            self.stds = np.load(self.std_path)
            self.type_num = self.centers.shape[0]

        def __call__(self, num_get=1):
            choice = my_rand(0, self.type_num - 1)
            color = self.centers[choice]
            std = self.stds[choice]
            choice_r = np.random.normal(color[0], std[0], num_get).astype(np.uint8)
            choice_g = np.random.normal(color[1], std[1], num_get).astype(np.uint8)
            choice_b = np.random.normal(color[2], std[2], num_get).astype(np.uint8)
            return choice_r, choice_g, choice_b
    def __init__(self, pr=1.0, target_label_list=None, num_line_range=None, pr_declay=0.7) -> None:
        if target_label_list is None:
            target_label_list = [i for i in range(1, 81)]
        if num_line_range is None:
            num_line_range = [0, 25]
        self.num_line_range = num_line_range
        self.pr = pr
        self.target_label_list = target_label_list
        self.temp_fill_color=(0,0,0)
        self.pr_declay=pr_declay
        self.debug_epoch=0
    def draw_noisy_line(self,image_, start_point_, end_point_, color, base_thickness, noise_range):
        # 计算线段的长度和方向
        length = int(np.sqrt((end_point_[0] - start_point_[0]) ** 2 + (end_point_[1] - start_point_[1]) ** 2))
        angle = np.arctan2(end_point_[1] - start_point_[1], end_point_[0] - start_point_[0])

        # 绘制每个线段
        for i in range(length):
            # 计算当前点的坐标
            x = int(start_point_[0] + i * np.cos(angle))
            y = int(start_point_[1] + i * np.sin(angle))

            # 在基础粗度上加上一个随机扰动
            thickness_ = base_thickness + my_rand(-noise_range, noise_range)
            if thickness_<=0:
                continue
            cv2.line(image_, (x, y), (x, y), color, thickness_)
    def get_line_points(self, image_shape, boxes_dict):
        random_idx=my_rand(0, len(boxes_dict)-1)
        selected_box=boxes_dict[random_idx]
        selected_point=(my_rand(selected_box["x"], selected_box["x"]+selected_box["w"]), my_rand(selected_box["y"], selected_box["y"]+selected_box["h"]))
        random_v1=my_rand(0, selected_point[0])
        random_v2=my_rand(selected_point[0]+1, image_shape[1])
        k=random.uniform(0,500)
        r_x1=ceil(random_v1)
        r_y1=ceil(selected_point[1]-k*(-random_v1+selected_point[0]))
        r_x2=ceil(random_v2)
        r_y2=ceil(selected_point[1]+k*(random_v2-selected_point[0]))
        r_x1=max(r_x1, 0)
        r_y1=max(r_y1, 0)
        r_x2=min(r_x2, image_shape[1]-1)
        r_y2=min(r_y2, image_shape[0]-1)
        return (r_x1, r_y1), (r_x2, r_y2)
    def debug(self, debug_image):
        cv2.imwrite("debug.jpg", debug_image)
    def __call__(self, image: np.ndarray, target: dict):
        global current_epoch 
        now_pr=self.pr*(self.pr_declay**current_epoch)
        if current_epoch!=self.debug_epoch:
            self.debug_epoch=current_epoch
            print(f"Epoch:{current_epoch}, now_pr:{now_pr}")
        if random.random() > now_pr:
            return image, target
        temp_image=np.zeros_like(image, dtype=np.uint8)
        white_rgb = (255, 255, 255)
        temp_image[:]=white_rgb
        cg = self.ColorGetter("/root/autodl-tmp/centers.npy", "/root/autodl-tmp/stds.npy")
        boxes = target["boxes"].copy()
        labels = target["labels"].copy()
        num_box = boxes.shape[0]
        found = False
        boxes_data = []
        for idx in range(num_box):
            if labels[idx] + 1 not in self.target_label_list:
                continue
            x1, y1, x2, y2 = boxes[idx]
            x = x1
            y = y1
            w = x2 - x1
            h = y2 - y1
            x = ceil(x)
            y = ceil(y)
            w = ceil(w)
            h = ceil(h)
            if h<=25 or w<=25:
                continue
            found = True
            boxes_data.append({"x": x, "y": y, "w": w, "h": h})
        if not found:
            return image, target
        sorted(boxes_data, key=lambda x: x["w"] * x["h"])
        smallest_box = boxes_data[0]
        self.num_line_range[1]=3*len(boxes_data) # TODO: 3 is a magic number to control the density of lines
        num_line=my_rand(self.num_line_range[0], self.num_line_range[1])
        largest_thickness=ceil(min(smallest_box["w"], smallest_box["h"])*0.6)
        base_thickness=ceil(largest_thickness*0.1)
        noise_range = 0 if base_thickness == 1 else base_thickness // 2
        for line_id in range(num_line):
            start_point, end_point=self.get_line_points(image.shape, boxes_data)
            if start_point==end_point:
                continue
            self.draw_noisy_line(temp_image, start_point, end_point, self.temp_fill_color, base_thickness, noise_range)
            num_points=temp_image.shape[0]*temp_image.shape[1]
            random_rs, random_gs, random_bs = cg(num_points)
            random_rs = np.reshape(random_rs, (temp_image.shape[0], temp_image.shape[1]))
            random_gs = np.reshape(random_gs, (temp_image.shape[0], temp_image.shape[1]))
            random_bs = np.reshape(random_bs, (temp_image.shape[0], temp_image.shape[1]))
            random_image = np.stack((random_rs, random_gs, random_bs), axis=-1)
            temp_image[np.all(temp_image == self.temp_fill_color, axis=-1)] = random_image[np.all(temp_image == self.temp_fill_color, axis=-1)]
        self.debug(temp_image)
        image[np.all(temp_image!=white_rgb, axis=-1)]=temp_image[np.all(temp_image!=white_rgb, axis=-1)]
        return image, target

[2.4871936568499184, 2.497301162037346, 2.435337257486224]
[87.34482758620689, 89.24137931034483, 82.96551724137932]
